In [1]:
from Bio.PDB import *
import numpy as np
import pandas as pd
import nglview as nv
import matplotlib.pyplot as plt
import pynmrstar
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from glob import glob
import os

# Idea

This analysis aims to find out the residue-type correlation between H-bond length (*actually, N---O distance*) and this amide's (HN and N) chemical shifts in the beta-sheet structures. We want to explain the diagonal tilts in hNH spectra of amyloids.

~~PIQCed PACSY is used as the source of chemical shifts and STRIDE class. (*alternatively: filter not by class but by angle???*)~~ PACSY is too small and inferior to the ReBoxitory data!

`biopython` package used to fetch and analyse PDB structures. 

## Algorithm
* For all PDBs:
    * run DSSP and identify sheets and strands (first chains only??..)
    * Find the matching BMRB file and take the shift data
    * List all amide nitrogens of the given residues
    * List all oxygens in the chain
    * Find pairs within 4 Angstrom radius

# 1. Reading DSSP tables and the corresponding BMRB assignments

## 1.1 Loading tables

In [2]:
wdir='~/sciebo/PDB analysis/'
dsspdir='C:/Users/Krairy/OneDrive/PhD/dssp/'

In [3]:
rel=pd.read_csv(wdir+'/PDB_BMRB_pairs.csv', index_col=0, sep=';')
rel=rel[~rel.index.duplicated(keep='first')].sort_index()

In [97]:
#rel.loc[5625][0]
rel

,bmrb_id
pdb_id,
1A2I,5625
1A3P,4201
1A5J,4114
1AXH,5702
1AYG,5086
...,...
7L53,30828
7L54,30829
7L55,30830


In [4]:
res_names = pd.read_csv("C:/Users/Krairy/OneDrive/PhD/H-bonds/rc_shifts.tab", sep='\t')
res_names.RES = res_names.RES.apply(lambda s: s.upper())
res_names = res_names.set_index('RES')['ID']
res_names

RES
ALA    A
CYS    C
ASP    D
GLU    E
PHE    F
GLY    G
HIS    H
ILE    I
LYS    K
LEU    L
MET    M
ASN    N
PRO    P
GLN    Q
ARG    R
SER    S
THR    T
VAL    V
TRP    W
TYR    Y
Name: ID, dtype: object

In [5]:
csv_dir = 'C:\\Users\\Krairy\\OneDrive\\PhD\\H-bonds\\DSSP_CSVs\\'

In [6]:
dssp_cols = "CHAIN AA SS ASA PHI PSI # NH-O1_idx NH-O1_En O-NH1_idx O-NH1_En NH-O2_idx NH-O2_En O-NH2_idx O-NH2_En".split()
## Note the order! The 5the col in the dictionary will be the res num. See Bio.PDB documentation (but double-check)

## 1.2 Combining the data
The following code executes about 40 minutes!

In [ ]:
## Takes about 40 minutes!!

fnames=list(glob(dsspdir+'*.dssp'))

seq_df = pd.DataFrame()
for fname in fnames:
    basename = os.path.basename(fname)
    pdb_id = basename.split('.')[0]
    bmrb_id = rel.loc[pdb_id.upper()][0]
    
    # Loading data
    
    ## DSSP
    dssp = make_dssp_dict(f"C:\\Users\\Krairy\\OneDrive\\PhD\\dssp\\{pdb_id}.dssp")
    dssp_df = pd.DataFrame.from_dict(dssp[0]).T
    dssp_df.reset_index(inplace=True)
    dssp_df.drop('level_1', axis=1, inplace=True)
    dssp_df.columns = dssp_cols
    dssp_df=dssp_df.loc[dssp_df.CHAIN == dssp_df.CHAIN.unique()[0]] # only the first chain!
    
    ## BMRB
    nmrstar = pynmrstar.Entry.from_database(bmrb_id)
    loops = nmrstar.get_saveframes_by_category('assigned_chemical_shifts')[0]
    shifts = loops['_Atom_chem_shift']
    data = StringIO(shifts.get_data_as_csv().replace("_Atom_chem_shift.", ""))
    df_bmrb = pd.read_csv(data, index_col=0)
    df_bmrb = df_bmrb.query('Atom_ID == "H" or Atom_ID == "N" or Atom_ID == "CA" or Atom_ID == "CB"')
    
    ## Assembley
    df_ = pd.DataFrame()
    df_ = dssp_df['CHAIN # AA SS PHI PSI NH-O1_idx NH-O1_En O-NH1_idx O-NH1_En NH-O2_idx NH-O2_En O-NH2_idx O-NH2_En'.split()]
    df_ = pd.merge(df_, df_bmrb['Seq_ID Comp_ID Atom_ID Val'.split()], 
                           left_on='#', right_on='Seq_ID' , how='outer') ## Here the magic happens!!
    
    ## Check for sequence match!!
    ### renaming cysteins (up to 4 CC bridges)
    if 'a' in df_.AA.unique():
        df_.loc['a' ,'AA'] = 'C'
    if 'b' in df_.AA.unique():
        df_.loc['b' ,'AA'] = 'C'
    if 'c' in df_.AA.unique():
        df_.loc['c' ,'AA'] = 'C'
    if 'd' in df_.AA.unique():
        df_.loc['d' ,'AA'] = 'C'
                    
    seq_df = df_[['AA', 'Comp_ID']]

    try:
        seq_df['AA_BMRB'] = df_['Comp_ID'].dropna().apply(lambda aa: res_names.loc[aa]) # to short format
    except KeyError: ## in case of non-standard amino acids
        continue ## skip the entry altogether
        
    seq_df = seq_df.dropna()
    
    if len(seq_df[['AA', 'AA_BMRB']].drop_duplicates(keep=False))==0:
        print('OK!')
        df_['BMRB_ID'] = bmrb_id
        df_['PDB_ID'] = pdb_id
        df_ = df_.query('SS == "E" or SS == "B"') # for the current research we need only Extended structures 
        # (and also we might need to save space on the drive)

        ## Writing to the disk to have a checkpoint
        df_.to_csv(csv_dir+f'/{pdb_id}_{bmrb_id}.csv')
    else:
        print('Too bad!')
        print(pdb_id, bmrb_id)

### Troubleshooting

Find the files with problems

In [205]:
def id_from_path(path):
    pdb_id = os.path.basename(path)
    pdb_id = pdb_id.split('.')[0].split('_')[0]
    return pdb_id

processed_IDs = [id_from_path(path) for path in list(glob(csv_dir+'/*.csv'))]
all_IDs = [id_from_path(path) for path in list(glob(dsspdir+'*.dssp'))]

PDBs_with_problems = set(all_IDs) - set(processed_IDs)
print(str(len(PDBs_with_problems))+" files had issues ("+str(len(PDBs_with_problems)/len(all_IDs)*100 )+ " %)")
print(str(len(all_IDs))+" structures in total")
PDBs_with_problems

575 files had issues (10.996366418053166 %)
5229 structures in total


{'1ee7',
 '1f2h',
 '1gh9',
 '1h3z',
 '1hbw',
 '1idh',
 '1ih9',
 '1je9',
 '1jjd',
 '1jwe',
 '1k09',
 '1k76',
 '1ka7',
 '1kfz',
 '1kgm',
 '1kn6',
 '1m4p',
 '1m4q',
 '1no8',
 '1nwb',
 '1nxn',
 '1nyn',
 '1oo9',
 '1osl',
 '1pxe',
 '1r05',
 '1r9k',
 '1rl1',
 '1rxr',
 '1s6l',
 '1s7e',
 '1so9',
 '1sp0',
 '1ssf',
 '1uvf',
 '1vj6',
 '1w1f',
 '1wa7',
 '1wh4',
 '1y4e',
 '1yel',
 '1z5f',
 '1z66',
 '1z8r',
 '1zac',
 '1zg2',
 '2a2b',
 '2a3s',
 '2aby',
 '2ami',
 '2bai',
 '2bug',
 '2c34',
 '2cef',
 '2ceh',
 '2cez',
 '2cfj',
 '2d2w',
 '2ddy',
 '2dii',
 '2fgx',
 '2fk4',
 '2fyj',
 '2g0k',
 '2g0l',
 '2g46',
 '2gg1',
 '2gjf',
 '2gov',
 '2h7d',
 '2h7e',
 '2hdl',
 '2hga',
 '2hgc',
 '2i94',
 '2i9y',
 '2ife',
 '2jgw',
 '2jgx',
 '2jm0',
 '2jmx',
 '2jn0',
 '2jng',
 '2jo0',
 '2jo9',
 '2joa',
 '2joh',
 '2jop',
 '2jpe',
 '2jqf',
 '2jrb',
 '2jt8',
 '2jvo',
 '2jw1',
 '2jx2',
 '2jzc',
 '2k17',
 '2k3b',
 '2k3m',
 '2k3u',
 '2k4b',
 '2k4j',
 '2k5w',
 '2k6m',
 '2k6n',
 '2k7z',
 '2k85',
 '2k8d',
 '2k8f',
 '2k9e',
 '2k9u',
 

## 1.4 Assembling into one dataset

In [206]:
df = pd.DataFrame() # the ultimate table
fnames=list(glob(csv_dir+'/*.csv'))

for fname in fnames:
    df_ = pd.read_csv(fname)
    if df_.empty:
        continue
    df = pd.concat((df, df_))

df   

,Unnamed: 0,CHAIN,#,AA,SS,PHI,PSI,NH-O1_idx,NH-O1_En,O-NH1_idx,...,NH-O2_idx,NH-O2_En,O-NH2_idx,O-NH2_En,Seq_ID,Comp_ID,Atom_ID,Val,BMRB_ID,PDB_ID
0,10,A,5,I,B,-79.4,123.6,11.0,-2.1,13,...,-3,-0.1,14,-0.4,5.0,ILE,N,122.800,5702,1axh
1,11,A,5,I,B,-79.4,123.6,11.0,-2.1,13,...,-3,-0.1,14,-0.4,5.0,ILE,H,9.190,5702,1axh
2,12,A,5,I,B,-79.4,123.6,11.0,-2.1,13,...,-3,-0.1,14,-0.4,5.0,ILE,CA,60.100,5702,1axh
3,13,A,5,I,B,-79.4,123.6,11.0,-2.1,13,...,-3,-0.1,14,-0.4,5.0,ILE,CB,40.200,5702,1axh
4,27,A,10,P,B,-75.7,128.2,0.0,0.0,25,...,0,0.0,-1,-0.1,10.0,PRO,CA,63.100,5702,1axh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,73,A,25,b,E,-110.5,146.7,-2.0,-0.4,-11,...,-11,-0.3,3,-0.1,25.0,CYS,N,117.007,30828,7l53
43,74,A,26,F,E,-131.3,95.1,-13.0,-2.7,-14,...,-2,-0.4,-13,-0.6,26.0,PHE,H,8.442,30828,7l53
44,75,A,26,F,E,-131.3,95.1,-13.0,-2.7,-14,...,-2,-0.4,-13,-0.6,26.0,PHE,CA,55.194,30828,7l53
45,76,A,26,F,E,-131.3,95.1,-13.0,-2.7,-14,...,-2,-0.4,-13,-0.6,26.0,PHE,CB,39.101,30828,7l53


In [207]:
df.to_csv(wdir+'/shifts_H-bonds.tab', sep='\t')

In [208]:
df.PDB_ID.unique().shape # How many structures without issues and with beta-sheets / strands we have

(3488,)

# 2. Measuring the H-bonds

In [19]:
# If already have some data:

df = pd.read_csv(wdir+'/shifts_H-bond_lens.tab', sep='\t')

In [25]:
df.loc[df.H_bond_1.notna()]

,PDB_ID,Seq_ID,level_0,index,Unnamed: 0,Unnamed: 0.1,CHAIN,#,AA,SS,...,O-NH2_En,Comp_ID,Atom_ID,Val,BMRB_ID,H_bond_1,H_bond_2,H_bond_11,H_bond_12,H_bond_min
0,1axh,5.0,0,0,0,10,A,5,I,B,...,-0.4,ILE,N,122.800,5702,3.182717,7.357613,2.879526,4.828830,2.879526
1,1axh,5.0,1,1,1,11,A,5,I,B,...,-0.4,ILE,H,9.190,5702,3.182717,7.357613,2.879526,4.828830,2.879526
2,1axh,5.0,2,2,2,12,A,5,I,B,...,-0.4,ILE,CA,60.100,5702,3.182717,7.357613,2.879526,4.828830,2.879526
3,1axh,5.0,3,3,3,13,A,5,I,B,...,-0.4,ILE,CB,40.200,5702,3.182717,7.357613,2.879526,4.828830,2.879526
4,1axh,10.0,4,4,4,27,A,10,P,B,...,-0.1,PRO,CA,63.100,5702,2.934207,2.934207,6.157717,5.939747,2.934207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324526,6e5c,76.0,324526,324526,177,273,A,76,R,E,...,-0.5,ARG,N,129.118,30495,2.899968,3.805474,2.904091,4.270998,NaN
324527,6e5c,77.0,324527,324527,178,274,A,77,E,E,...,-0.2,GLU,H,8.877,30495,2.908913,4.246701,3.112823,6.909288,NaN
324528,6e5c,77.0,324528,324528,179,275,A,77,E,E,...,-0.2,GLU,CA,55.382,30495,2.908913,4.246701,3.112823,6.909288,NaN
324529,6e5c,77.0,324529,324529,180,276,A,77,E,E,...,-0.2,GLU,CB,33.346,30495,2.908913,4.246701,3.112823,6.909288,NaN


In [8]:
#df = pd.read_csv(wdir+'/shifts_H-bonds.tab', sep='\t')
#df.loc[:, ["NH-O1_idx", "NH-O2_idx", "O-NH1_idx", "O-NH2_idx"]].fillna(0, inplace=True)

In [194]:
fnames=list(glob(csv_dir+'/*.csv'))

for fname in fnames[:3]:
    basename = os.path.basename(fname)
    basename = basename.split('.')[0].split('_')
    pdb_id, bmrb_id = basename
    print(pdb_id, bmrb_id)

1a5j 4114
1axh 5702
1azp 5905


In [252]:
pdbl.retrieve_pdb_file(pdb_id.upper(), pdir=pdbdir)
structure = parser.get_structure("protein", pdbdir+pdb_id+".cif")
try:
    chain = structure[0]["A"]
except KeyError:
    problems.append(pdb_id)
        
chain[int(5)]['N'] - chain[5+11]['O']

Structure exists: 'D://Structures/1axh.cif' 


3.182717

In [33]:
df

level_0   index  Unnamed: 0  Unnamed: 0.1 CHAIN   # AA SS  \
PDB_ID Seq_ID                                                              
1axh   5.0           0       0           0            10     A   5  I  B   
       5.0           1       1           1            11     A   5  I  B   
       5.0           2       2           2            12     A   5  I  B   
       5.0           3       3           3            13     A   5  I  B   
       10.0          4       4           4            27     A  10  P  B   
...                ...     ...         ...           ...   ...  .. .. ..   
7l53   25.0     352758  352758          42            73     A  25  b  E   
       26.0     352759  352759          43            74     A  26  F  E   
       26.0     352760  352760          44            75     A  26  F  E   
       26.0     352761  352761          45            76     A  26  F  E   
       26.0     352762  352762          46            77     A  26  F  E   

                 PHI    PSI  ...  O-NH2_En  Comp_ID  Atom_ID      Val  \
PDB_ID Seq_ID                ...                                        
1axh   5.0     -79.4  123.6  ...      -0.4      ILE        N  122.800   
       5.0     -79.4  123.6  ...      -0.4      ILE        H    9.190   
       5.0     -79.4  123.6  ...      -0.4      ILE       CA   60.100   
       5.0     -79.4  123.6  ...      -0.4      ILE       CB   40.200   
       10.0    -75.7  128.2  ...      -0.1      PRO       CA   63.100   
...              ...    ...  ...       ...      ...      ...      ...   
7l53   25.0   -110.5  146.7  ...      -0.1      CYS        N  117.007   
       26.0   -131.3   95.1  ...      -0.6      PHE        H    8.442   
       26.0   -131.3   95.1  ...      -0.6      PHE       CA   55.194   
       26.0   -131.3   95.1  ...      -0.6      PHE       CB   39.101   
       26.0   -131.3   95.1  ...      -0.6      PHE        N  118.525   

               BMRB_ID  H_bond_1  H_bond_2  H_bond_11 H_bond_12 H_bond_min  
PDB_ID Seq_ID                                                               
1axh   5.0        5702  3.182717  7.357613   2.879526  4.828830   2.879526  
       5.0        5702  3.182717  7.357613   2.879526  4.828830   2.879526  
       5.0        5702  3.182717  7.357613   2.879526  4.828830   2.879526  
       5.0        5702  3.182717  7.357613   2.879526  4.828830   2.879526  
       10.0       5702  2.934207  2.934207   6.157717  5.939747   2.934207  
...                ...       ...       ...        ...       ...        ...  
7l53   25.0      30828  4.462179  6.515833   6.450683  6.365672        NaN  
       26.0      30828  2.782624  4.425014   2.692164  3.928674        NaN  
       26.0      30828  2.782624  4.425014   2.692164  3.928674        NaN  
       26.0      30828  2.782624  4.425014   2.692164  3.928674        NaN  
       26.0      30828  2.782624  4.425014   2.692164  3.928674        NaN  

[352763 rows x 27 columns]

In [32]:
pdbdir='D://Structures/' # to drop PDBs to - careful! Will download a lot of data!

df = df.reset_index()
df = df.set_index(['PDB_ID'])

parser = MMCIFParser()
pdbl = PDBList()
problems = []  

for pdb_id in df.index.unique()[3480:]:
    
    df = df.reset_index()
    df = df.set_index(['PDB_ID'])
    
    pdbl.retrieve_pdb_file(pdb_id.upper(), pdir=pdbdir)
    structure = parser.get_structure("protein", pdbdir+pdb_id+".cif")
    try:
        chain = structure[0]["A"]
    except KeyError:
        problems.append(pdb_id)
        continue

    # Measuring the H-bonds which were identified by DSSP 
    residues = list(df.loc[pdb_id, 'Seq_ID'].unique())
    
    df = df.reset_index()
    df = df.set_index(['PDB_ID', 'Seq_ID'])

    for i_num in residues:
        #if df.loc[pdb_id, i_num].loc[:, 'NH-O1_idx'].unique()[0] == 0:
        #    continue
        #else:
        ## considering N-H--O bonds only    
        partner1 = i_num+int(df.loc[(pdb_id, i_num),'NH-O1_idx'].fillna(0).unique()[0])
        partner2 = i_num+int(df.loc[(pdb_id, i_num), 'NH-O2_idx'].fillna(0).unique()[0])
        partner11 = i_num+int(df.loc[(pdb_id, i_num),'O-NH1_idx'].fillna(0).unique()[0])
        partner12 = i_num+int(df.loc[(pdb_id, i_num),'O-NH2_idx'].fillna(0).unique()[0])
        try:
            Ni = chain[int(i_num)]['N']
            O1 = chain[int(partner1)]['O']
            O2 = chain[int(partner2)]['O']

            Oi = chain[int(i_num)]['O']
            N1 = chain[int(partner11)]['N']
            N2 = chain[int(partner12)]['N']

            H_bond1_d = Ni-O1
            H_bond2_d = Ni-O2

            H_bond11_d = Oi-N1
            H_bond12_d = Oi-N2

            df.loc[(pdb_id, i_num), 'H_bond_1'] = H_bond1_d
            df.loc[(pdb_id, i_num), 'H_bond_2'] = H_bond2_d
            df.loc[(pdb_id, i_num), 'H_bond_11'] = H_bond11_d
            df.loc[(pdb_id, i_num), 'H_bond_12'] = H_bond12_d
        except:
            problems.append(pdb_id)
            continue
    df.to_csv(wdir+'/shifts_H-bond_lens_DSSPs.tab', sep='\t') 

Structure exists: 'D://Structures/7jn6.cif' 


C:\Users\Krairy\anaconda3\lib\site-packages\pandas\core\indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2974: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\async_helpers.py:78: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


C:\Users\Krairy\anaconda3\lib\site-packages\pandas\core\indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2974: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\async_helpers.py:78: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


C:\Users\Krairy\anaconda3\lib\site-packages\pandas\core\indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2974: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\async_helpers.py:78: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


C:\Users\Krairy\anaconda3\lib\site-packages\pandas\core\indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2974: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\async_helpers.py:78: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


C:\Users\Krairy\anaconda3\lib\site-packages\pandas\core\indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2974: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\async_helpers.py:78: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


C:\Users\Krairy\anaconda3\lib\site-packages\pandas\core\indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2974: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\async_helpers.py:78: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


C:\Users\Krairy\anaconda3\lib\site-packages\pandas\core\indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2974: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\async_helpers.py:78: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


C:\Users\Krairy\anaconda3\lib\site-packages\pandas\core\indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2974: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
C:\Users\Krairy\anaconda3\lib\site-packages\IPython\core\async_helpers.py:78: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


In [30]:
l=list(df.reset_index().PDB_ID.unique())
#l=list(df.PDB_ID.unique())
l.index('7jn6')

## 1909

3480

In [19]:
df.columns

Index(['Seq_ID', 'index', 'Unnamed: 0', 'Unnamed: 0.1', 'CHAIN', '#', 'AA',
       'SS', 'PHI', 'PSI', 'NH-O1_idx', 'NH-O1_En', 'O-NH1_idx', 'O-NH1_En',
       'NH-O2_idx', 'NH-O2_En', 'O-NH2_idx', 'O-NH2_En', 'Comp_ID', 'Atom_ID',
       'Val', 'BMRB_ID', 'H_bond_1', 'H_bond_2', 'H_bond_11', 'H_bond_12'],
      dtype='object')

In [34]:
df.to_csv(wdir+'/shifts_H-bond_lens_DSSPs.tab', sep='\t') 

In [408]:
set(problems)

{'2e8j',
 '2epq',
 '2epr',
 '2eps',
 '2ers',
 '2exd',
 '2f09',
 '2f1e',
 '2f3v',
 '2f3w',
 '2f65',
 '2f8b',
 '2fek',
 '2ffk',
 '2ffw',
 '2fho',
 '2fin',
 '2fj3',
 '2fo8',
 '2fvn',
 '2fws',
 '2fwu',
 '2fy9',
 '2g0q',
 '2g1e',
 '2g35',
 '2g5m',
 '2ggr',
 '2ghf',
 '2gjh',
 '2gmo',
 '2gow',
 '2grg',
 '2gs0',
 '2gw6',
 '2h0p',
 '2h7t',
 '2hfq',
 '2hj8',
 '2hst',
 '2htf',
 '2hug',
 '2hv1',
 '2i5o',
 '2i83',
 '2i85',
 '2ifs',
 '2ikd',
 '2ike',
 '2in2',
 '2ivw',
 '2iz4',
 '2jmm',
 '2jnp',
 '2job',
 '2jom',
 '2jox',
 '2jpd',
 '2jpi',
 '2jpp',
 '2jqg',
 '2jqz',
 '2jra',
 '2jrl',
 '2jsd',
 '2jsn',
 '2jt5',
 '2jt6',
 '2jub',
 '2jug',
 '2juo',
 '2jv2',
 '2jve',
 '2jvf',
 '2jvr',
 '2jvv',
 '2jw8',
 '2jwe',
 '2jwn',
 '2jx9',
 '2jxa',
 '2jxo',
 '2jxx',
 '2jxy',
 '2jya',
 '2jzd',
 '2k01',
 '2k0d',
 '2k16',
 '2k1b',
 '2k1g',
 '2k29',
 '2k2a',
 '2k2j',
 '2k2m',
 '2k2o',
 '2k31',
 '2k3g',
 '2k3j',
 '2k3r',
 '2k3t',
 '2k4d',
 '2k4t',
 '2k4z',
 '2k5c',
 '2k5j',
 '2k5v',
 '2k60',
 '2k6a',
 '2k6d',
 '2k6q',
 

# 3. Neighbor search

In [31]:
df.loc[df.H_bond_1.notna()].PDB_ID.unique()

array(['1axh', '1azp', '1azq', ..., '2lp6', '2lp7', '2lpd'], dtype=object)

In [11]:
# If we already have some data:

df = pd.read_csv(wdir+'/shifts_H-bond_lens_DSSP_PDBs_1.tab', sep='\t')
l=list(df.PDB_ID.unique())
#l=list(df.PDB_ID.unique())
#l.index('7jn6')

In [8]:
df

,Unnamed: 0,PDB_ID,Seq_ID,level_0,index,Unnamed: 0.1,Unnamed: 0.1.1,CHAIN,#,AA,...,H_bond_12,H_bond_min,O_1,NO_len_1,O_2,NO_len_2,O_3,NO_len_3,O_4,NO_len_4
0,0,1axh,5.0,0,0,0,10,A,5,I,...,4.828830,2.879526,3.0,3.698927,16.0,3.182717,NaN,NaN,NaN,NaN
1,1,1axh,5.0,1,1,1,11,A,5,I,...,4.828830,2.879526,3.0,3.698927,16.0,3.182717,NaN,NaN,NaN,NaN
2,2,1axh,5.0,2,2,2,12,A,5,I,...,4.828830,2.879526,3.0,3.698927,16.0,3.182717,NaN,NaN,NaN,NaN
3,3,1axh,5.0,3,3,3,13,A,5,I,...,4.828830,2.879526,3.0,3.698927,16.0,3.182717,NaN,NaN,NaN,NaN
4,4,1axh,10.0,4,4,4,27,A,10,P,...,5.939747,2.934207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352758,352758,7l53,25.0,352758,352758,42,73,A,25,b,...,6.365672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352759,352759,7l53,26.0,352759,352759,43,74,A,26,F,...,3.928674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352760,352760,7l53,26.0,352760,352760,44,75,A,26,F,...,3.928674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352761,352761,7l53,26.0,352761,352761,45,76,A,26,F,...,3.928674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 3.1 Adding hydrogens with `pdbtools`  - need `pymol` license!

In [33]:
#import pymol2.PyMOL as pymol
import pdbtools

for pdb_id in df.loc[df.H_bond_1.notna()].PDB_ID.unique():
    pymol.cmd.load(pdbdir+pdb_id+".cif", 'myprotein') # file
    # pymol.cmd.fetch('👾👾👾👾', 'myprotein') # PDB code
    pymol.cmd.h_add()
    pymol.cmd.save(pdbdir+pdb_id+".cif")

ModuleNotFoundError: No module named 'pymol2.PyMOL'

## 3.2 Extracting the N---O distances

In [ ]:
############# WARNING! Creates A TON OF text output! ###############################
############## The notebook can become unopenable! #################################

radius = 3.9 #Angstrom for an H-bond

pdbdir='D://Structures/' # to drop PDBs to - careful with disc size!

#df = df.reset_index()
#df = df.set_index(['PDB_ID'])

parser = MMCIFParser()
pdbl = PDBList()
problems = []  

search_set = []

loop_counter=0

## For every already processed PDB:
for pdb_id in df.loc[df['H_bond_1'].notna()].set_index('PDB_ID').index.unique()[2700:3000]:
    print(pdb_id, l.index(pdb_id))

    df = df.set_index(['PDB_ID'])
    
    #pdbl.retrieve_pdb_file(pdb_id.upper(), pdir=pdbdir)
    structure = parser.get_structure("_", pdbdir+pdb_id+".cif")
    
    #Everything calculated only for chain A
    try:
        chain = structure[0]["A"]
    except KeyError:
        problems.append(pdb_id)
        df = df.reset_index()
        continue
        
    
    # Collecting all residues of the current protein 
    residues = list(df.loc[pdb_id, '#'].unique())
    
    O_atom_list = []
    for res in structure[0]["A"]:
        if res.has_id("O"):              # Surprisingly, not every residue has an oxygen
            O_atom_list.append(res["O"]) # Gathering all oxygens in the structure
        else:
            continue

    for rnum in residues:
        
        # Pool of atoms
        search_set = O_atom_list
        
        ## adding nitrogens to the pool
        try:
            search_set.append(chain[int(rnum)]['N'])
        except ValueError:
            problems.append(pdb_id)
            continue
        except KeyError:
            problems.append(pdb_id)
            continue
        
        # Get pairs of neighbors from the pool
        H_bond_finder=NeighborSearch(search_set)
        raw_out = H_bond_finder.search_all(radius=radius, level='R')
        
        df = df.reset_index().set_index(['PDB_ID', '#'])
        
        i=0
        for pair in raw_out:
            print(pair)
            for res in pair:
                if res.get_id()[1] == rnum:
                    
                    # Potential partners found
                    ## Initialize atoms
                    if pair[0].get_id()[1] == rnum:
                        partner_id = pair[1].get_id()[1]
                        n = pair[0]['N'] # our N
                        o = pair[1]['O'] # other's O
                    else:
                        partner_id = pair[0].get_id()[1]
                        n = pair[1]['N'] # our N
                        o = pair[0]['O'] # other's O

                    # Exclude direct sequential neighbors!
                    print('Residue:', rnum, 'Partner', partner_id)
                    if abs(partner_id-rnum) < 2:
                        continue
                    else:
                        dist = n - o
                        print(dist)
                        if dist < radius:
                            # Found an H-bond! Now write it down
                            i+=1 
                            df.loc[(pdb_id, rnum), f'O_{i}'] = partner_id
                            df.loc[(pdb_id, rnum), f'NO_len_{i}'] = dist
                        
    loop_counter+=1
    df = df.reset_index()
    if loop_counter==10:
        loop_counter=0
        print('writing')
        # Regularly update the file
        ## Every 10 loops
        df.to_csv(wdir+'/shifts_H-bond_lens_DSSP_PDBs.tab', sep='\t') 
        print('written')
print(problems)

In [ ]:
df.loc[df['H_bond_1'].notna()].set_index('PDB_ID').index.unique()[2349]

In [19]:
df.to_csv(wdir+'/shifts_H-bond_lens_DSSP_PDBs.tab', sep='\t') 

## 3.3. Including info about the direct neighbors

In [ ]:
#df = pd.read_csv(wdir+'/shifts_H-bond_lens_DSSP_PDBs_neighbors.tab', sep='\t') 
#df = df.reset_index()
pdbdir='D://Structures/' # to drop PDBs to - careful with disc size!

#df = df.reset_index()
#df = df.set_index(['PDB_ID'])

parser = MMCIFParser()
pdbl = PDBList()

loop_counter=0
for pdb_id in df.loc[df['H_bond_1'].notna()].set_index('PDB_ID').index.unique():

    print(pdb_id)
    df = df.set_index(['PDB_ID'])
    structure = parser.get_structure("_", pdbdir+pdb_id+".cif")
    
    residues = list(df.loc[pdb_id, 'Seq_ID'].dropna().unique())
    
    df = df.reset_index().set_index(['PDB_ID', 'Seq_ID'])
    
    for rnum in residues:
        prev_num = int(rnum - 1)
        next_num = int(rnum + 1)
        
        try: 
            df.loc[(pdb_id, rnum), 'prev_type'] = structure[0]['A'][prev_num].get_resname()
        except KeyError:
            df.loc[(pdb_id, rnum), 'prev_type'] = 'X'
        try:
            df.loc[(pdb_id, rnum), 'next_type'] = structure[0]['A'][next_num].get_resname()
        except KeyError:
            df.loc[(pdb_id, rnum), 'next_type'] = 'X'
    
    loop_counter+=1
    df = df.reset_index()
    if loop_counter==20:
        loop_counter=0
        print('writing')
        # Regularly update the file
        df.to_csv(wdir+'/shifts_H-bond_lens_DSSP_PDBs_neighbors.tab', sep='\t') 
        print('written')

In [41]:
df.to_csv(wdir+'/shifts_H-bond_lens_DSSP_PDBs_neighbors.tab', sep='\t') 

### Writing the clean table

In [34]:
df[['PDB_ID', 'BMRB_ID', 'CHAIN', '#', 'AA', 'Comp_ID', 'SS', 'prev_type', 'next_type', 'PHI', 'PSI', 'NH-O1_idx',
       'NH-O1_En', 'O-NH1_idx', 'O-NH1_En', 'NH-O2_idx', 'NH-O2_En',
       'O-NH2_idx', 'O-NH2_En', 'H_bond_1', 'H_bond_2', 'H_bond_11', 'H_bond_12', 'H_bond_min', 
       'O_1', 'NO_len_1', 'O_2', 'NO_len_2', 'O_3', 'NO_len_3', 'O_4', 'NO_len_4', 
        'Atom_ID', 'Val']].to_csv(wdir+'/shifts_H-bond_lens_DSSP_PDBs_neighbors.tab', sep='\t') 

----